In [8]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersORM, ResumesORM, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

#### Без relationship

In [9]:
with session_factory() as session:
    query = (
        select(WorkersORM)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2024-05-24 09:33:26,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-24 09:33:26,337 INFO sqlalchemy.engine.Engine SELECT workers.id, workers.username 
FROM workers 
 LIMIT %(param_1)s::INTEGER
2024-05-24 09:33:26,339 INFO sqlalchemy.engine.Engine [cached since 346.5s ago] {'param_1': 2}
result_orm=[<models.WorkersORM object at 0x0000014D62F86650>, <models.WorkersORM object at 0x0000014D62F86590>]
result_dto=[WorkersDTO(username='Jack', id=1), WorkersDTO(username='Misha', id=2)]
2024-05-24 09:33:26,340 INFO sqlalchemy.engine.Engine ROLLBACK


#### C relationship

In [12]:
with session_factory() as session:
    query = (
        select(WorkersORM)
        .options(selectinload(WorkersORM.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

AttributeError: type object 'WorkersORM' has no attribute 'resumes'

#### JOIN

In [10]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

In [11]:
with session_factory() as session:
    query = (
        select(
            ResumesORM.workload,
            func.avg(ResumesORM.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesORM)
        .filter(and_(
            ResumesORM.title.contains("Python"),
            ResumesORM.compensation > 40000,
        ))
        .group_by(ResumesORM.workload)
        .having(func.avg(ResumesORM.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

2024-05-24 09:33:59,850 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-24 09:33:59,854 INFO sqlalchemy.engine.Engine SELECT resumes.workload, CAST(avg(resumes.compensation) AS INTEGER) AS avg_compensation 
FROM resumes 
WHERE (resumes.title LIKE '%%' || %(title_1)s::VARCHAR || '%%') AND resumes.compensation > %(compensation_1)s::INTEGER GROUP BY resumes.workload 
HAVING avg(resumes.compensation) > %(avg_1)s::INTEGER
2024-05-24 09:33:59,855 INFO sqlalchemy.engine.Engine [generated in 0.00095s] {'title_1': 'Python', 'compensation_1': 40000, 'avg_1': 70000}
result_orm=[]
result_dto=[]
2024-05-24 09:33:59,862 INFO sqlalchemy.engine.Engine ROLLBACK
